# A (simplified) search with mainstream software

In this tutorial, we will run a full search for pulsations in real data and obtain the orbital solution of an unknown binary pulsar.



## Exercise 1: low-DM pulsar

Enter the directory `data/lowdm`

```
$ cd data/lowdm
```

You will find a file named `guppi_55757_B1508+55_0004_0001.fil`. The source in these data is a very well-known pulsar, B1508+55, with a sharp profile and low dispersion measure. Data were collected with the [Green Bank Telescope](https://science.nrao.edu/facilities/gbt/), at 820 Hz, with a total band of 200 MHz split in 1024 channels.

For a rough periodicity search, we do not need all this information. Therefore, as a first step we sum up all these data in a single channel.

```
$ prepdata guppi_55757_B1508+55_0004_0001.fil -o 1508
```
This will create two files: `1508.dat`, containing the light curve, and `1508.inf`, a text file with some information about the observation. Let us see what the Fourier transform (see first lesson, this is actually the absolute value of the Fourier transform, or the square root of the PDS) looks like.

```
$ realfft 1508.dat
$ explorefft 1508.fft


      Interactive FFT Explorer
         by Scott M. Ransom
            October, 2001

 Button or Key       Effect
 -------------       ------
 Left Mouse or I     Zoom in  by a factor of 2
 Right Mouse or O    Zoom out by a factor of 2
 Middle Mouse or D   Show details about a selected frequency
 <                   Shift left  by a full screen width
 >                   Shift right by a full screen width
 ,                   Shift left  by 1/8 of the screen width
 .                   Shift right by 1/8 of the screen width
 + or =              Increase the power scale (make them taller)
 - or _              Decrease the power scale (make them shorter)
 S                   Scale the powers automatically
 N                   Re-normalize the nowers by one of several methods
 P                   Print the current plot to a file
 G                   Go to a specified frequency
 L                   Load a zaplist showing potential RFI locations
 Z                   Add a frequency chunk to the RFI zaplist
 H                   Show the harmonics of the center frequency
 ?                   Show this help screen
 Q                   Quit

Examining B1508+55 data from '1508.fft'.

Memory mapping the input FFT.  This may take a while...

```
The second command will open a PGPLOT window where you will be able to zoom and look at all the periodicities contained in the data. The mouse and keyboard keys to touch to perform the various actions (zoom, scroll, etc) are listed in the terminal.

You will notice that the fundamental frequency of the pulsation (~5 Hz) is not the most prominent peak in the FT. This is due to the shape of the pulse.
Now, we can use a program that automatically looks at the FT and performs quality checks of the signals contained in it, ranking them (ignore the `-zmax` switch for now, we do that to limit the search to the routine methods and not the advanced ones).

```
$ accelsearch 1508.fft -zmax 0 -numharm 16
```


```
prepfold -accelcand 1 -accelfile 1508_ACCEL_0.cand guppi_55757_B1508+55_0004_0001.fil
```


![1508](1508.png)

## Exercise 2: high-DM pulsar


*TBD*

## Exercise 3: binary pulsar.

In the directory `data/split` you will find 20 observations, 3 ks each, of a simulated binary pulsar.

### 3.1 Finding the local period and pdot in each observation

We start from an estimate of the period: 0.05124 s. We run an accelerated epoch folding search in each file around this period. The switch `-p` suggests a starting period for the search; `-mjds -events -double` are there only because of the format of the data; `-npfact 10` extends the range of periods and pdots we will be using for the search by a factor 10.
```
$ prepfold psr_0.events -p 0.05124 -mjds -events -double -npfact 10
$ prepfold psr_1.events -p 0.05124 -mjds -events -double -npfact 10
(...)
```
or, to make it quick,
```
$ for i in psr_*.events; do prepfold $i -p 0.05124 -mjds -events -double -npfact 10; done
```




### 3.2. Approximate values for the orbital modulation
Plot the best solutions seen so far (`-pb 1 -T0 0 -x 1` is just telling `fitorb.py` to use those starting values for the orbital period, the ascending node passage and the projected semi-major axis respectively.):
```
$ fitorb.py psr*.bestprof -p 0.05124 -pb 1 -T0 0 -x 1
```
We see a modulation with a period of ~2.3 days. Let's play using different starting values for the parameters until the fit of the parameters is good. `fitorb.py` outputs a series of parameters like
```
PEPOCH      49354.042438
P0  0.0512450030268001
BINARY BT
A1   5.00038618340817
E                   0
T0  -0.469972316212605
PB   2.30000329860082
OM                  0
``` 
Let us copy and paste this output to a new file called `fit.par`.

### 3.3a. "Freezing" the local solutions
**Attention**: This step is overly complicated (having to use different approximate parameter files for each chunk, instead of the approximate orbital solution found above for all) because of a bug in PRESTO. I filed an Issue to try to correct it.

To calculate TOAs, PRESTO wants bestprofs obtained without searching. So, we have to rerun `prepfold` giving it already the best parameters found in the previous search, and telling it not to perform additional searches. This can be done in many ways. 

1. Easy but possibly inconsistent(?)
One is to read in each bestprof the lines
```
# P_bary (ms)      =  51.240437722057   +/- 1.25e-06
# P'_bary (s/s)    =  2.33518439161511e-10 +/- 3.24e-12
# P''_bary (s/s^2) =  0                 +/- 7e-15
```
and calling `prepfold` with the options
```
$ prepfold psr_0.events -p 0.051240437722057 -pdot 2.33518439161511e-10 -pddot 0 -nosearch -mjds -events -double -npfact 10
```
The drawback of this option is that _some .bestprof files will be overwritten, others not_ (because of how the period is used in the file name), and it will be more difficult 

2. Slightly trickier, but surely consistent
In step 1 we obtained local best values for $p$, $\dot{p}$ and $\ddot{p}$ in each observation. For each observation file, we have a corresponding file that ends in `.bestprof` and contains these best values. For each of these local solutions, we create a file with the same name but extension `.par`, copying the values of each parameter and converting to the relevant units, as follows:
\begin{align}
\nu &= \frac{1}{p} \\
\dot{\nu} &= -\frac{1}{p^2}\dot{p} \\
\ddot{\nu} &= \frac{2}{p^3}\dot{p} - \frac{1}{p^2}\ddot{p} \\
\end{align}
So, the parameter file will look like
```
PSRJ         FAKE                # Just any name, but it must be here!
PEPOCH       49353.232270000000  # In the bestprof: Epoch_bary (MJD) =  49353.232270000000
F0           0.051251497947019   # Convert to Hz the line 
                                 # P_bary (ms)      =  51.251497947019  +/- 1.39e-06 
F1           4.46613404291e-08   # Convert to Hz/s the period line plus 
                                 # P'_bary (s/s)    =  -1.17312659353888e-10 +/- 3.44e-12 
F2           2.0445609693e-16    # Convert to Hz/s^2 the period and pdot lines, plus
                                 # P''_bary (s/s^2) =  0                 +/- 7.44e-15
```
Refold, this time with the `-timing` option
```
$ prepfold psr_0.events -timing psr_0_51.24ms_Cand.pfd.par -mjds -events -double
$ prepfold psr_1.events -timing psr_1_51.24ms_Cand.pfd.par -mjds -events -double
(...)
```
or, more rapidly
```
$ for i in psr*.events; do bestprof=\${i/.events/}*.bestprof; echo $bestprof ; \
    prepfold $i -timing ${bestprof/.bestprof/.par} -mjds -events -double; done
```

### 3.3b. Calculate TOAs
Now, to calculate TOAs we use the `get_TOA.py` script in PRESTO, with the following syntax
```
get_TOAs.py -n 1 -e -g 0.2 psr_1_PSR_FAKE.pfd.par
```
`-n 1` says that we are calculating one TOA per file; `-e` is again related to the original file format; `-g 0.2` means to use a Gaussian with width 0.2.
The TOAs will be printed in the screen:
```
$ get_TOAs.py -n 1 -e -g 0.2 psr_0_PSR_51.24.pfd
/psr/Soft/presto/lib/python/psr_utils.py:669: RuntimeWarning: invalid value encountered in divide
  DM/(0.000241*freq_emitted*freq_emitted), 0.0)
@                  0.000 49353.0180863062417  1785.39
```
You can ignore the first warning message. Beware, instead, of error or warning messages about FFTFIT failing to do something. They often mean that you have bad statistics in your data or the template you chose is not adequate.
Copy all lines starting by "@" to a file called `all.tim`

### 3.4. Refine the solution with Tempo2

*TBD*